# Library

In [1]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("Yarn").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

8


In [3]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Function

In [4]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part, "DAYS_EMPLOYED_vect")       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
## !! mind the fn NOT TO rewrite results
fn = '8cpu_by_12n__8m_yarn_ssd_repart'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50, 100]
partitions = [8, 12, 16, 24, 36, 72] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 8 



=== size_mult=1;         n_part_req=8; n_part_act=8; df_count=252137 ===

model 0 build time 7.0 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9fc4c863286d, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 1 build time 4.35 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_8d5a1ab56ea6, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 2 build time 3.98 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_1124f765e59b, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=12; n_part_act=12; df_count=252137 ===

model 0 build time 5.11 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_06b36774c047, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 1 build time 5.01 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5478081bbb39, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 5.86 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_ec66c7941687, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=16; n_part_act=16; df_count=252137 ===

model 0 build time 4.85 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_bb2348ef1edf, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 1 build time 4.81 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5184f902ea85, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 2 build time 4.81 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3e146121923f, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=24; n_part_act=24; df_count=252137 ===



model 0 build time 6.09 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_ceb39fae9930, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 1 build time 6.14 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_7d2e0394d2de, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 5.84 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_20e938d51014, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=36; n_part_act=36; df_count=252137 ===



...

model 2 build time 12.01 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_7dd97685a5fe, depth=12, numNodes=45, numClasses=2, numFeatures=1


=== size_mult=100;         n_part_req=72; n_part_act=72; df_count=25213700 ===



model 0 build time 16.24 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_bc8e2539669c, depth=11, numNodes=37, numClasses=2, numFeatures=1


model 1 build time 12.57 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b068b6947708, depth=11, numNodes=37, numClasses=2, numFeatures=1


model 2 build time 11.12 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_06f71545d618, depth=11, numNodes=37, numClasses=2, numFeatures=1


# Results

In [6]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.median(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_9fc4c863286d,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_8d5a1ab56ea6,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_1124f765e59b,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_06b36774c047,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_5478081bbb39,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_ec66c7941687,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_bb2348ef1edf,', 'depth=14,', 'numNodes=123,', 'numC

,8,12,16,24,36,72
1,4.3,5.1,4.8,6.1,8.0,13.3
2,4.5,5.9,5.7,7.9,9.3,21.8
5,6.1,8.0,7.8,10.2,13.8,21.7
10,8.6,7.4,6.0,5.4,6.2,7.9
15,9.2,7.3,6.3,6.2,6.2,8.3
20,9.9,7.6,7.1,6.8,6.4,7.9
30,11.6,9.8,8.6,7.4,7.0,8.5
50,16.4,12.5,10.4,9.6,7.8,9.2
100,28.6,21.2,15.1,12.7,12.0,12.6


,8,12,16,24,36,72
1,123.0,123.0,123.0,123.0,123.0,123.0
2,145.0,129.0,131.0,137.0,135.0,135.0
5,139.0,141.0,115.0,127.0,51.0,95.0
10,55.0,85.0,107.0,73.0,53.0,69.0
15,59.0,113.0,57.0,49.0,77.0,61.0
20,71.0,65.0,41.0,77.0,37.0,73.0
30,49.0,41.0,43.0,57.0,37.0,35.0
50,39.0,35.0,47.0,43.0,29.0,47.0
100,33.0,41.0,41.0,37.0,45.0,37.0


In [7]:
#fn = '4cpu_by_8n__4m_yarn_ssd'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
print(fn)
tmp

8cpu_by_12n__8m_yarn_ssd_repart


,Unnamed: 0,8,12,16,24,36,72
0,1,4.3,5.1,4.8,6.1,8.0,13.3
1,2,4.5,5.9,5.7,7.9,9.3,21.8
2,5,6.1,8.0,7.8,10.2,13.8,21.7
3,10,8.6,7.4,6.0,5.4,6.2,7.9
4,15,9.2,7.3,6.3,6.2,6.2,8.3
5,20,9.9,7.6,7.1,6.8,6.4,7.9
6,30,11.6,9.8,8.6,7.4,7.0,8.5
7,50,16.4,12.5,10.4,9.6,7.8,9.2
8,100,28.6,21.2,15.1,12.7,12.0,12.6


In [8]:
#fn = '4cpu_by_8n__4m_yarn_ssd'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
print(fn)
tmp

8cpu_by_12n__8m_yarn_ssd_repart


,Unnamed: 0,8,12,16,24,36,72
0,1,123.0,123.0,123.0,123.0,123.0,123.0
1,2,145.0,129.0,131.0,137.0,135.0,135.0
2,5,139.0,141.0,115.0,127.0,51.0,95.0
3,10,55.0,85.0,107.0,73.0,53.0,69.0
4,15,59.0,113.0,57.0,49.0,77.0,61.0
5,20,71.0,65.0,41.0,77.0,37.0,73.0
6,30,49.0,41.0,43.0,57.0,37.0,35.0
7,50,39.0,35.0,47.0,43.0,29.0,47.0
8,100,33.0,41.0,41.0,37.0,45.0,37.0
